# Kelvin Abrokwa-Johnson
## homework 9

## 12.2.16

Use the results of Exercise 15 to approximate the solution to

$$\frac{\delta u}{\delta t} - \frac{\delta^2 u}{\delta x^2} = 2, 0 < x < 1, 0 < t;$$
$$u(0, t) = u(1, t) = 0, 0 < t$$
$$u(x, 0) = sin \pi x + x(1 - x)$$

with $h=0.1$, $k = 0.01$. Compare your answer at $t = 0.25$ to the actual solution $u(x,t) = e^{-\pi x} sin \pi x + x(1 - x)$.

Crank-Nicolson Implementation

In [88]:
function getA(lam, n)
    out = zeros(n,n)
    for i = 1:n
        out[i, i] = 1 + lam
        for j = 1:n
            if i == j-1 || i == j+1
                out[i, j] = -lam/2
            end
        end
    end
    return out
end

function getB(lam, n)
    out = zeros(n,n)
    for i = 1:n
        out[i, i] = 1 - lam
        for j = 1:n
            if i == j-1 || i == j+1
                out[i, j] = lam/2
            end
        end
    end
    return out
end

function CrankNicolson(a, k, h, m, t, f, F)
    w = zeros(m)
    x = zeros(m)
    lam = a^2*(k/h^2)
    A = getA(lam, m)
    B = getB(lam, m)
    for i = 1:m
        x[i] = i*h
        w[i] = f(x[i])
    end
    for i = 1:t/k
        for j = 1:m
            w[j] = w[j] + F(i*h)*k / 2
        end
        w[:] = A\(B*w[:])
        for j = 1:m
            w[j] = w[j] + F(i*h)*k / 2
        end
    end
    return w;
end;

Backward Difference Implementation

In [89]:
function GenA(lam, n)
    out = zeros(n, n)
    for i = 1:n
        out[i, i] = 1 + 2*lam
        for j = 1:n
            if i == j-1 || i == j+1
                out[i, j] = -lam
            end
        end
    end
    return out
end

function BackwardDiff(a, k, h, m, t, f, F)
    w = zeros(m)
    x = zeros(m)
    lam = a^2*(k/h^2)
    A = GenA(lam, m)
    for i = 1:m
        x[i] = i*h
        w[i] = f(x[i])
    end
    for i = 1:t/k
        for j = 1:m
            w[j] = w[j] + F(i*h)*k
        end
        w[:] = A\w[:]
    end
    return x, w
end;

In [90]:
function F(x)
    return 2
end
f = x -> sin(pi*x) + x*(1-x)
a = 1
k = 0.01
h = 0.1
m = 9
t = 0.25;

Backward Difference results

In [91]:
x, w1 = BackwardDiff(a, k, h, m, t, f, F);
w1

9-element Array{Float64,1}:
 0.119926
 0.216922
 0.288346
 0.332101
 0.346841
 0.332101
 0.288346
 0.216922
 0.119926

Crank-Nicolson results

In [92]:
w2 = CrankNicolson(a, k, h, m, t, f, F);
w2

9-element Array{Float64,1}:
 0.116689
 0.210766
 0.279874
 0.322141
 0.336369
 0.322141
 0.279874
 0.210766
 0.116689

In [93]:
# actual answer
function u(x, t, m)
    w = zeros(m)
    for i = 1:m
        w[i] = (e^(-pi^2*t))*sin(pi*x[i]) + x[i]*(1-x[i])
    end
    return w
end;
w3 = u(x, t, m);
w3

9-element Array{Float64,1}:
 0.116206
 0.209847
 0.278609
 0.320654
 0.334805
 0.320654
 0.278609
 0.209847
 0.116206

In [94]:
println("Backwards Difference error: ", norm(w1 - w3));
println("Crank-Nicolson difference error: ", norm(w2 - w3));

Backwards Difference error: 0.026913370911504836
Crank-Nicolson difference error: 0.003496388432786654
